# 当数据很多时，内存资源显得十分宝贵，合理利用内存非常重要

In [1]:
import datetime
import os
import threading
from pymongo import MongoClient

In [2]:
threadNum = 128
folderNum = 8
osm = MongoClient('192.168.168.7', 5077)
au = osm.admin.authenticate("sa","6hNmEVCd",mechanism="SCRAM-SHA-1")
db = osm.aisdata
col = db.shipInfo
erCol = db.shipInfoError

In [3]:
DATA = "/root/ais/data/2018/trajectory%s/" %( folderNum )  #数据文件
TASK = "/root/ais/Projects/month%s/" %( folderNum )      #工作路径

#  1.分割任务
 **将每月的数据分为foldernum份，每一份又分为threadNum份**

In [4]:
tasksplit = 10        #分割为10个文件夹
createVar = locals()
dataDir = DATA
taskListDir = TASK
taskdir = os.path.join(taskListDir, "folder%02d" % (folderNum)) 
Var = locals()
filePath = []

## 扫描所有数据文件

In [5]:
if os.path.exists(taskdir):
    print("spliting files finashed.\n")
else:
    os.mkdir(taskdir)
for root, dirs, files in os.walk(dataDir):      #扫描所有数据文件名
    for file in files:  
        if file.endswith('txt'):
            filePath.append(os.path.join(root, file))
print("All txt: %d" % ( len(filePath) ))

spliting files finashed.

All txt: 84990


## 将所有将要插入的数据分为tasksplit个

In [6]:
for i in range(tasksplit):          #生成tasksplit个list 
    queue = "%02d" % ( i )
    createVar["fileQueue" + queue] = []    

In [7]:
for i in range(len(filePath)):      #分配task给每一个tasksplit
    queue = "%02d" % (i % tasksplit)
    createVar["fileQueue" + queue].append(filePath[i])

In [8]:
for i in range(tasksplit):          #打印出每个tasksplit长度
    queue = "%02d" % ( i )
    print(len(createVar["fileQueue" + queue]))

8499
8499
8499
8499
8499
8499
8499
8499
8499
8499


## 将每个tasksplit分为threadNum份

In [9]:
for i in range(tasksplit):          #生成文件夹
    queue = "%02d" % ( i )
    taskdirnum = os.path.join(taskdir,"Task%s"%(queue))
    if not os.path.isdir(taskdirnum):
        os.mkdir(taskdirnum)

In [10]:
for m in range(threadNum*tasksplit):          #256*10个Var[]
    Vqu = "%02d" % ( m )
    Var["fqueue" + Vqu] = []
for j in range(tasksplit):      #0
    ques = "%02d" % ( j )         #0
    cVarlen = len(createVar["fileQueue" + ques])   #16330
    queVar = j*threadNum   # 256*j 
    for k in range(cVarlen):      #16330
        Vqu = k % threadNum
        Varque = "%02d" % (Vqu+queVar)    #64
        Var["fqueue" + Varque].append(createVar["fileQueue" + ques][k])    #64*256

In [11]:
len(Var["fqueue25"])

KeyError: 'fqueue2559'

## 分配所有任务到每一个文件夹里

In [13]:
# MAIN
START = datetime.datetime.now()
strSTART = START.strftime("%Y-%m-%d %H:%M:%S")
with open (os.path.join(TASK, "%02d_time.log" % ( folderNum )), "a") as l:
    l.write(strSTART + "\n")
#######################################################################
TOTAL = 0
for i in yield_insert(1):
    print(i)
######################################################################    
END = datetime.datetime.now()
strEND = END.strftime("%Y-%m-%d %H:%M:%S")
with open (os.path.join(TASK, "%02d_time.log" % ( folderNum )), "a") as lg:
    lg.write(strEND)
DELTA = (END - START).seconds
print("All threads finished, used %s seconds." % (DELTA))
print("%d pieces of data (%d) has been inserted, %d errors happened." %( TOTAL, col.find().count(), erCol.find().count() ))

Task00 folder task00.txt has 66 pieces task.
Task00 folder task01.txt has 66 pieces task.
Task00 folder task02.txt has 66 pieces task.
Task00 folder task03.txt has 66 pieces task.
Task00 folder task04.txt has 66 pieces task.
Task00 folder task05.txt has 66 pieces task.
Task00 folder task06.txt has 66 pieces task.
...trajectory8 thread 71 assigned 66 pieces of data.
trajectory8 thread 71 assigned 3 pieces of data.
trajectory8 thread 71 assigned 110808 pieces of data.
trajectory8 thread 121 assigned 3 pieces of data.
trajectory8 thread 121 assigned 2 pieces of data.
trajectory8 thread 121 assigned 2 pieces of data.


trajectory8 thread 59 assigned 10128 pieces of data.
trajectory8 thread 4 assigned 2 pieces of data.
trajectory8 thread 4 assigned 1608 pieces of data.
trajectory8 thread 4 assigned 18 pieces of data.
trajectory8 thread 4 assigned 2 pieces of data.
trajectory8 thread 117 assigned 3 pieces of data.
trajectory8 thread 117 assigned 3 pieces of data.
trajectory8 thread 117 assigned 125 pieces of data.
trajectory8 thread 117 assigned 3 pieces of data.
trajectory8 thread 98 assigned 3 pieces of data.
trajectory8 thread 98 assigned 2 pieces of data.
trajectory8 thread 98 assigned 3 pieces of data.
trajectory8 thread 127 assigned 296 pieces of data.
trajectory8 thread 127 assigned 2 pieces of data.
trajectory8 thread 123 assigned 2 pieces of data.
trajectory8 thread 123 assigned 2 pieces of data.
trajectory8 thread 123 assigned 2 pieces of data.
trajectory8 thread 123 assigned 2 pieces of data.
trajectory8 thread 52 assigned 2 pieces of data.
trajectory8 thread 120 assigned 2 pieces of data.
t

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


trajectory8 thread 71 assigned 2 pieces of data.
trajectory8 thread 3 assigned 2 pieces of data.
trajectory8 thread 11 assigned 39372 pieces of data.
trajectory8 thread 11 assigned 2 pieces of data.
trajectory8 thread 11 assigned 2 pieces of data.
trajectory8 thread 11 assigned 3 pieces of data.


### for在数据非常多时很占内存，然后用yeild生成器执行十次main， 不报错，但是发现服务器一条数据也没插入，莫名其妙

In [7]:
def yield_insert(n):
    for i in range(n):
        yield main(i)
    print("all data have inserted") 
    print("end")

### 下面是为断点续插，插入进度可视化的探索，但是不能插入数据，弃用了

In [ ]:
class threadInsert(threading.Thread):
    def __init__(self, queue, fileList, col, erCol, strtask):
        threading.Thread.__init__(self)
        self.queue = queue
        self.fileList = fileList
        self.col = col
        self.erCol = erCol
        self.strtask = strtask

    def run(self):
        total = 0
        with open(self.fileList, "r") as taskList:
            taskList = taskList.read().split("\n")
            
        for txt in taskList:
            if txt == "":
                continue
            else:
                with open(txt, "r") as data:
                    data = data.readlines()
                month = strtask + " "+ os.path.dirname(txt)[-11:]
                print("%s thread %s assigned %s pieces of data." % (month, self.queue, len(data) ))
                total += len(data)

                col_insert = []
                erCol_insert = []
                for c in data:
                    a = insertData(c, self.col, self.erCol, col_insert, erCol_insert)

        self.result = total

    def getResult(self):
        try:
            return self.result
        except:
            return Exception


In [ ]:
START = datetime.datetime.now()
strSTART = START.strftime("%Y-%m-%d %H:%M:%S")

with open (os.path.join(TASK, "%02d_time.log" % ( folderNum )), "a") as tlog:
    tlog.write(strSTART + "\n")
############################################

TOTAL = 0
#for j in range(threadNum):
TaskFolder = os.path.join(TASK, "folder%02d" %(folderNum), "Task00")
threads = []
try:
    for i in range(threadNum):
        queue = "%02d" % ( i )
        taskFile = os.path.join(TaskFolder, "task%s.txt" % ( queue ))
        strtask = "Task%02d "%(j) + taskFile[-10:]
        t = threadInsert(queue=i, fileList=taskFile, col=col, erCol=erCol, strtask=strtask)
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
        result = t.getResult()
        TOTAL += result
        print("finished %d pieces of data." % ( result ))
except:
    CUR = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(os.path.join(TASK, "error.log"), "a") as erLog:
        erLog.write(CUR + "\n")
        erLog.write("%02d" % ( folderNum ) + " - " + str(Exception) + "\n")
#############################################  

END = datetime.datetime.now()
strEND = END.strftime("%Y-%m-%d %H:%M:%S")

with open (os.path.join(TASK, "%02d_time.log" % ( folderNum )), "a") as tlog:
    tlog.write(strEND)
    
DELTA = (END - START).seconds

print("All threads finished, used %s seconds." % (DELTA))
print("%d pieces of data (%d) has been inserted, %d errors happened." %( TOTAL, col.find().count(), erCol.find().count() ))

In [ ]:
START = datetime.datetime.now()
strSTART = START.strftime("%Y-%m-%d %H:%M:%S")
with open (os.path.join(TASK, "%02d_time.log" % ( folderNum )), "w") as tlog:
    tlog.write(strSTART + "\n")

Total = 0
try:
    for j in range(16, threadNum):
        threads = []
        TOTAL = 0
        startque = datetime.datetime.now()
        queues = "%02d" % ( j )
        taskFile = os.path.join(TASK, "folder%02d" %(folderNum), "task%s.txt" % ( queues ))
#        if os.path.exist(taskFile):
        strtask = taskFile[-10:]
        for i in range(threadNum):
            t = threadInsert(queue=i, fileList=taskFile, col=col, erCol=erCol, strtask=strtask)
            t.start()
            threads.append(t)
        os.remove(taskFile)
        costque = (datetime.datetime.now() - startque).seconds
        for t in threads:
            t.join()
            result = t.getResult()
            TOTAL += result
        Total += TOTAL
        rstr = strtask[:-4] + "finished %d pieces of data." % ( TOTAL )
        with open (os.path.join(TASK, "%02d_time.log" % ( folderNum )), "a") as l:
            l.write(rstr + "cost %s seconds "%(costque) + "\n")
        # print("%d pieces of data finished, %d errors." % ( col.find().count(), erCol.find().count() ))

except:
    CUR = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(os.path.join(TASK, "error.log"), "a") as erLog:
        erLog.write(CUR + "\n")
        erLog.write("%02d" % ( folderNum ) + " - " + str(Exception) + "\n")

END = datetime.datetime.now()
strEND = END.strftime("%Y-%m-%d %H:%M:%S")
with open (os.path.join(TASK, "%02d_time.log" % ( folderNum )), "a") as tlog:
    tlog.write(strEND)
    
DELTA = (END - START).seconds
print("All threads finished, used %s seconds." % (DELTA))
print("%d pieces of data (%d) has been inserted, %d errors happened." %( Total, col.find().count(), erCol.find().count() ))


In [ ]:
def initTask(dataDir, taskListDir):
    taskdir = os.path.join(taskListDir, "folder%02d" % (folderNum))   #/root/ais/Projects/month%d/folder%d
    if os.path.exists(taskdir):
        print("spliting files finashed.\n")
        return

    filePath = []       #保存所有txt文件的绝对路径

    for root, dirs, files in os.walk(dataDir):      #扫描所有数据文件名
        for file in files:  
            if file.endswith('txt'):
                filePath.append(os.path.join(root, file))   

    print("All txt: %d" % ( len(filePath) ))

    for i in range(threadNum):          #循环执行所有线程
        queue = "%02d" % ( i )
        createVar["fileQueue" + queue] = []    #生成threadNum个list 保存filepath
        
    for i in range(len(filePath)):      #将所有txt文件分为 Threadnum 份
        queue = "%02d" % (i % threadNum)
        createVar["fileQueue" + queue].append(filePath[i])

    for i in range(threadNum):          #
        queue = "%02d" % ( i )
        if not os.path.isdir(taskdir):           #/root/ais/Projects/month%d/folder%d/Task%s/ 不存在则创建
            os.mkdir(taskdir)
        tasknum = os.path.join(taskdir,"Task%s"%(queue))
        if not os.path.isdir(tasknum):
            os.mkdir(tasknum)
            #文件夹task00:
            
        for j in range(threadNum):
            que = "%02d" % ( j )
#            if not os.path.isdir(taskdir,"Task%s"%(queue)):     #/root/ais/Projects/month%d/folder%d/Task%s/ 不存在则创建
 #               os.mkdir(taskdir,"Task%s"%(queue))
                
            fn = os.path.join(taskdir ,"Task%s"%(queue),"task%s.txt" % (que))    #folder%d/task%s.txt
            if os.path.isfile(fn):                              #删除目录下已存在的task%s.txt文件
                os.remove(fn)
                
            for m in range(threadNum):          #
                Queue = "%02d" % ( m )
                Var["fileQueue" + Queue] = []
                #128*Var[] = createVar[]
                
            for k in range(len(createVar["fileQueue" + queue])):      #createVar[00] 
                qu = "%02d" % (k % threadNum)
                Var["fileQueue" + qu].append(createVar["fileQueue" + queue][k])
                
            with open(fn, "a") as task:                   #将createVar["fileQueue" + queue]写入task%s.txt
                for file in Var["fileQueue" + que]:      
                    task.write(file + "\n")

            print("Task%s task%s: %d" % (queue, que, len(Var["fileQueue" + que])))
            Var["fileQueue" + que] = []
            fn = []
        
    # release memory        
    createVar["fileQueue" + queue] = []

    print("spliting files finished.\n")

    # release memory
    filePath = []

